## Produce Driver Standings

In [0]:
dbutils.widgets.text('p_file_date', '2021-03-21')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

Find race years for which the data is to be reprocessed

In [0]:
race_results_list = spark.read.table('formula1.gold.race_results')\
                                .filter(f"file_date = '{v_file_date}'") \
                                .select('race_year') \
                                .distinct().collect()
race_results_list = [item.race_year for item in race_results_list]

In [0]:
from pyspark.sql.functions import sum, count, when, col
race_results_df = spark.read.table('formula1.gold.race_results') \
                            .filter(col('race_year').isin(race_results_list))
display(race_results_df)

In [0]:
driver_standings_df = race_results_df \
                    .groupBy("race_year", "driver_name", "driver_nationality") \
                    .agg(sum("points").alias('total_points'), count(when(col("position") == 1, True)).alias('wins'))
display(driver_standings_df)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

driver_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'), desc("wins"))
final_df = driver_standings_df.withColumn('rank', rank().over(driver_rank_spec))

In [0]:
display(final_df)

In [0]:
#incremental_load(final_df, 'f1_presentation', 'driver_standings', 'race_year')

In [0]:
merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_year = src.race_year"
partition_column = 'race_year'
merge_delta_data(final_df, 'formula1', 'gold', 'driver_standings', merge_condition, partition_column)